# Cassandra database population for Sparkify

This jupyter notebook loads Sparkify user data in from a CSV an creates 3 tables that can be used for analytical purposes. 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating a Cluster

In [2]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

# Create KeySpace
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = {
            'class': 'SimpleStrategy',
            'replication_factor': 1
        }
        """)
except Exception as e:
    print(e)
    
# Set KEYSPACE to the keyspace specified above
session.set_keyspace("sparkify")

# Load in the data 
data = pd.read_csv('event_datafile_new.csv')

# Table 1: Items in Session

Should allow queries like: 
- Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession  = 4


In [3]:
# drop table if it already exists
session.execute("""
    DROP TABLE IF EXISTS item_in_session_table
""")

# create the table
session.execute("""
    CREATE TABLE IF NOT EXISTS item_in_session_table 
    (
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        length decimal,
        PRIMARY KEY (sessionId, itemInSession)
    )
""")

# insert data into table
for i, row in data.iterrows():
    query = 'INSERT INTO item_in_session_table (sessionId, itemInSession, artist, song, length) VALUES (%s, %s, %s, %s, %s)'
    session.execute(query, (row['sessionId'], row['itemInSession'], row['artist'], row['song'], row['length']))

# verify the data was entered into the table
result = session.execute("SELECT * FROM item_in_session_table WHERE sessionId=338 AND itemInSession=4")
for row in result:
    print(row)

Row(sessionid=338, iteminsession=4, artist='Faithless', length=Decimal('495.3073'), song='Music Matters (Mark Knight Dub)')


# Table 2: User session history

This table should provide answers for queries like:
- Give me only the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


In [4]:

# Drop table if it exists
session.execute("""
    DROP TABLE IF EXISTS user_session_history
""")

# Create table for query 
session.execute("""
    CREATE TABLE IF NOT EXISTS user_session_history 
    (
        userId int,
        sessionId int,
        itemInSession int,
        song text,
        artist text,
        firstName text,
        lastName text,
        PRIMARY KEY (userId, sessionId, itemInSession)
    )
""")

# insert data into table
for i, row in data.iterrows():
    query2 = """
        INSERT INTO user_session_history (userId, sessionId, itemInSession, song, artist, firstName, lastName) 
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    session.execute(query2, (row['userId'], row['sessionId'], row['itemInSession'], row['song'], row['artist'], row['firstName'], row['lastName']))
    

# verify the data was entered into the table
result = session.execute("SELECT * FROM user_session_history WHERE sessionId=182 AND userId=10", timeout=None)
for row in result:
    print(row)

Row(userid=10, sessionid=182, iteminsession=0, artist='Down To The Bone', firstname='Sylvie', lastname='Cruz', song="Keep On Keepin' On")
Row(userid=10, sessionid=182, iteminsession=1, artist='Three Drives', firstname='Sylvie', lastname='Cruz', song='Greece 2000')
Row(userid=10, sessionid=182, iteminsession=2, artist='Sebastien Tellier', firstname='Sylvie', lastname='Cruz', song='Kilometer')
Row(userid=10, sessionid=182, iteminsession=3, artist='Lonnie Gordon', firstname='Sylvie', lastname='Cruz', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


# Table 3: Who listened to which song?

This table should be able to answer questions like: 
 - give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [5]:

# drop table if exists
session.execute("""
    DROP TABLE IF EXISTS who_listened_to_which_song
""")

# create table with required columns
session.execute("""
    CREATE TABLE IF NOT EXISTS who_listened_to_which_song
    (
        song text,
        firstName text,
        lastName text,
        PRIMARY KEY (song, firstName, lastName)
    )
""")

# insert data into table
for i, row in data.iterrows():
    query3 = """
        INSERT INTO who_listened_to_which_song (song, firstName, lastName) 
        VALUES (%s, %s, %s)
    """
    session.execute(query3, (row['song'], row['firstName'], row['lastName']))
    
# verify data has been added correctly
result = session.execute("SELECT * FROM who_listened_to_which_song WHERE song='All Hands Against His Own'", timeout=None)
for row in result:
    print(row)

Row(song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')
Row(song='All Hands Against His Own', firstname='Tegan', lastname='Levine')


### Drop the tables before closing out the sessions

In [6]:
session.execute("DROP TABLE IF EXISTS item_in_session_table")
session.execute("DROP TABLE IF EXISTS user_session_history")
session.execute("DROP TABLE IF EXISTS table_query3")

### Close the session and cluster connection¶

In [7]:
session.shutdown()
cluster.shutdown()